In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
from catboost.utils import eval_metric
import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
df = pd.read_csv("train_lt.csv")
df.drop('PRODUCT_RK', axis=1, inplace=True)
df.drop('PRODUCT_RK_y', axis=1, inplace=True)
df.drop('STORE_LOCATION_RK', axis=1, inplace=True)
df['is_train'] = ~df.demand.isna() # mark non-nan demand as train observations

In [3]:
# TODO: think about filling NaN

In [4]:
# for each product this fills NaNs with the mean specific for this product and store
for product in tqdm(df['product_rk'].unique()):
    t = df[df['product_rk'] == product]
    for store in t['store_location_rk'].unique():
        g = t[t['store_location_rk'] == store]
        g['PRICE_REGULAR'] = g['PRICE_REGULAR'].fillna(g["PRICE_REGULAR"].mean())
        g['PRICE_AFTER_DISC'] = g['PRICE_AFTER_DISC'].fillna(g["PRICE_AFTER_DISC"].mean())
        t[t['store_location_rk'] == store] = g
    
    df[df['product_rk'] == product] = t

100%|██████████| 2137/2137 [02:39<00:00, 13.37it/s]


In [5]:
# for those rows that are still empty we fillna by mean demand for the product (without store)
for product in tqdm(df['product_rk'].unique()):
    t = df[df['product_rk'] == product]
    t['PRICE_REGULAR'] = t['PRICE_REGULAR'].fillna(g["PRICE_REGULAR"].mean())
    t['PRICE_AFTER_DISC'] = t['PRICE_AFTER_DISC'].fillna(g["PRICE_AFTER_DISC"].mean())
    df[df['product_rk'] == product] = t

100%|██████████| 2137/2137 [00:38<00:00, 55.04it/s]


In [6]:
df['demand'] = df['demand'].fillna(-1).astype(int) # fill nan demand as -1 to avoid crashing
df['DISCOUNT'] = df['PRICE_AFTER_DISC'] / df['PRICE_REGULAR']
df['is_discount'] = df['DISCOUNT'] != 1 

# TODO: натренить на обычно лаге в текущем году. когда предсказываем -> идем последовательно от января до декабря.

In [7]:
df.to_csv("data_fixed.csv", index=False) # checkpoint

In [9]:
df['date'] = pd.to_datetime(df['date'])
df['week_number'] = df['date'].dt.isocalendar().week

In [10]:
df['years'] = df['date'].dt.year
df['months'] = df['date'].dt.month

In [37]:
def add_newyear(df):
    df['NewYear'] = 0
    t = df[(df['years'].isin([2016, 2017, 2018, 2019]))&(df['week_number'].isin([51, 52]))]
    t['NewYear'] = 1
    df[(df['years'].isin([2016, 2017, 2018, 2019]))&(df['week_number'].isin([51, 52]))] = t
    return df

def add_feb14(df):
    df['is_feb14'] = 0
    t = df[(df['years'].isin([2016]))&(df['week_number'].isin([6]))]
    t['is_feb14'] = 1
    df[(df['years'].isin([2016]))&(df['week_number'].isin([6]))] = t
    
    t = df[(df['years'].isin([2017, 2018]))&(df['week_number'].isin([6, 7]))]
    t['is_feb14'] = 1
    df[(df['years'].isin([2017, 2018]))&(df['week_number'].isin([6, 7]))] = t
    
    t = df[(df['years'].isin([2019]))&(df['week_number'].isin([7]))]
    t['is_feb14'] = 1
    df[(df['years'].isin([2019]))&(df['week_number'].isin([7]))] = t
    return df

def add_march8(df):
    df['is_march8'] = 0
    t = df[(df['years'].isin([2016, 2017, 2018, 2019]))&(df['week_number'].isin([9, 10]))]
    t['is_march8'] = 1
    df[(df['years'].isin([2016, 2017, 2018, 2019]))&(df['week_number'].isin([9, 10]))] = t
    return df

def add_september1(df):
    df['is_september1'] = 0
    t = df[(df['years'].isin([2016, 2017, 2018, 2019]))&(df['week_number'].isin([35]))]
    t['is_september1'] = 1
    df[(df['years'].isin([2016, 2017, 2018, 2019]))&(df['week_number'].isin([35]))] = t
    return df

def add_blackFR(df):
    df['is_blackFR'] = 0
    t = df[(df['years'].isin([2016, 2017, 2018]))&(df['week_number'].isin([47]))]
    t['is_blackFR'] = 1
    df[(df['years'].isin([2016, 2017, 2018]))&(df['week_number'].isin([47]))] = t
    
    t = df[(df['years'].isin([2019]))&(df['week_number'].isin([48]))]
    t['is_blackFR'] = 1
    df[(df['years'].isin([2019]))&(df['week_number'].isin([48]))] = t
    return df

In [38]:
df = add_newyear(df)
df = add_feb14(df)
df = add_march8(df)
df = add_september1(df)
df = add_blackFR(df)

In [43]:
df.to_csv("data_holidays.csv", index=False) # checkpoint

In [176]:
df = pd.read_csv('data_holidays.csv')

In [177]:
is_test = [0]*len(df) # creating is_test columns

In [178]:
smp = pd.read_csv("sample_lt.csv") # reading sample

In [179]:
for idd in smp['index']: # mark those rows that appear in the test as is_test=1
    is_test[idd] = 1

In [180]:
df['is_test'] = is_test # save this column

In [181]:
# remove rows which are not in the test or train. (Maybe there are rows with NaN demand that are not in the test)
df = df[(df['is_test']==1)|(df['is_train']==1)] 

In [182]:
df = df.drop('date', axis=1) # remove unnecessary date

In [ ]:
# TODO: skip next step

In [61]:
# remove products that are not presented in the test 
# (this didn't decreased or increased score on Kaggle, maybe shouldn't remove in the future)
test_prods = df[df['is_test']==1].product_rk.unique() 
df = df[df['product_rk'].isin(test_prods)]

In [183]:
df['weeklag1'] = 0
df['weeklag2'] = 0
df['yearlag1'] = 0

In [184]:
final_df = pd.DataFrame()
ii = 0
for product in tqdm(df['product_rk'].unique()): # run over unique products
    ii += 1
    temp_arr = []
    prod_df = df[df['product_rk'] == product]
    # TODO: calculate lags for stores
    for index, row in prod_df.iterrows():
        product_rk = row['product_rk']
        weeknum = row['week_number']
        year = row['years']
        year = int(year)

        # weeklag from prev year
        for i in [1, 2]: # 1, 2 since we only have weeklag1, weeklag2
            new_weeknum = weeknum - i
            new_year = year - 1
            if new_weeknum < 0: # if it was first week of the year we might get week 0 or -1
                new_weeknum = new_weeknum % 52 + 1
                new_year = year - 1
            # looking for lags a year ago
            lag = prod_df[(prod_df['years']==(new_year))&(prod_df['week_number']==(new_weeknum))] 
            if len(lag) >= 1: # lags found
                lag = lag['demand'].mean() # TODO: median
            elif len(lag) == 0: # no lags
                lag = 0
            elif lag['demand'].values[0] == -1: # if demand is NaN
                lag = 0
            row[f'weeklag{i}'] = lag

        #yearlag
        for i in [1]:
            lag = prod_df[(prod_df['years']==(year-1))&(prod_df['week_number']==(weeknum))]
            if len(lag) >= 1:
                lag = lag['demand'].mean() # TODO: median
            elif len(lag) == 0:
                lag = 0
            elif lag['demand'].values[0] == -1:
                lag = 0
            row[f'yearlag{i}'] = lag
        temp_arr.append(row)
        
    tt = pd.DataFrame.from_dict(temp_arr)
    final_df = final_df.append(tt)

100%|██████████| 2137/2137 [09:51<00:00,  3.61it/s]


In [124]:
df = final_df.copy()

In [125]:
df.to_csv('data+lags.csv', index=False) # checkpoint

## Training

In [126]:
df['SKU'] = df['product_rk'] * 100000 + df['store_location_rk'] # creating SKU column for unique pairs

In [127]:
uniq_skus = df[df['is_test']==0]['SKU'].unique()

In [128]:
len(uniq_skus) # number of unique pairs

11550

In [129]:
val_skus = uniq_skus[::4] # take 1/4 or 25% for validation

In [130]:
len(val_skus)

2888

In [131]:
X = df[(df['is_train']==1)].drop(['id', 'demand', 'is_train', 
                                  'is_test', 'SKU', 'years'], axis=1)
y = df[df['is_train']==1]['demand']

# it is train, SKU not in validation and year is < 2018
X_train = df[(df['is_train']==1)&(~df['SKU'].isin(val_skus))&(df['years']<2018)].drop(['id', 'demand', 
                                                                                        'is_train', 'is_test', 
                                                                                        'SKU', 'years'], 
                                                                                        axis=1)
y_train = df[(df['is_train']==1)&(~df['SKU'].isin(val_skus))&(df['years']<2018)]['demand']

# it is train, SKU in validation and year is 2018
X_test = df[(df['is_train']==1)&(df['SKU'].isin(val_skus))&(df['years']==2018)].drop(['id', 'demand', 
                                                                                       'is_train', 'is_test', 
                                                                                       'SKU', 'years'], 
                                                                                       axis=1)
y_test = df[(df['is_train']==1)&(df['SKU'].isin(val_skus))&(df['years']==2018)]['demand']

In [132]:
cat_features = ['product_rk', 'PROMO1_FLAG', 'is_blackFR', 'is_feb14', 'store_location_rk',
                'is_march8', 'NewYear', 'is_september1', 'PROMO2_FLAG', 'AUTORIZATION_FLAG', 'is_discount']

In [99]:
# my own grid search
for ls in ['MAE']:
    for leaf in [7, 9]: # 7, 9, 11
        for iterations in [1000]:
            for depth in [4, 6, 8]:
                lr = 0.1
                cat = CatBoostRegressor(loss_function=ls, # TODO: try another losses
                                        eval_metric='SMAPE',
                                        learning_rate=lr,
                                        metric_period=100,
                                        l2_leaf_reg=leaf,
                                        depth=depth,
                                        iterations=iterations, random_state=0)

                cat.fit(X_train, y_train, cat_features, verbose=False)
                preds = cat.predict(X_test)
                preds = preds.clip(min=0)
                res = eval_metric(y_test.values, preds, 'SMAPE')
                print('leaf:', leaf, ', lr:', lr, ', depth:', depth, ', iterations:', iterations, ls, ', res:', res)

In [163]:
# base model
cat = CatBoostRegressor(loss_function='MAE', # TODO: try another losses
                        eval_metric='SMAPE',
                        learning_rate=0.1,
                        metric_period=100,
                        iterations=500,
                        l2_leaf_reg=4,
                        depth=5,
                        random_state=0)

cat.fit(X_train, y_train, cat_features)

0:	learn: 196.6543601	total: 25.2ms	remaining: 12.6s
100:	learn: 171.4214431	total: 1.38s	remaining: 5.43s
200:	learn: 169.5746620	total: 2.61s	remaining: 3.89s
300:	learn: 169.0284715	total: 4.14s	remaining: 2.73s
400:	learn: 168.4988283	total: 5.74s	remaining: 1.42s
499:	learn: 168.3547374	total: 6.94s	remaining: 0us


In [164]:
preds = cat.predict(X_test)
preds = preds.clip(min=0)

In [165]:
eval_metric(y_test.values, preds, 'SMAPE')

[146.79064233851966]

In [166]:
# try this one :D
eval_metric(y_test.values, preds.astype(int), 'SMAPE')

[55.87276156335722]

In [167]:
for i in range(len(cat.feature_importances_)):
    print(cat.feature_importances_[i], cat.feature_names_[i])

31.02590742053221 product_rk
15.306320013202267 store_location_rk
7.284049009536575 PROMO1_FLAG
0.053359530577006035 PROMO2_FLAG
0.3698406680333062 PRICE_REGULAR
0.23419497106647302 PRICE_AFTER_DISC
0.1994755222935547 NUM_CONSULTANT
2.8160063510257864 AUTORIZATION_FLAG
21.596744750718074 DISCOUNT
0.1874866205015876 is_discount
9.735960937379746 week_number
6.281238082766466 months
4.606499327287438 NewYear
7.21108522715423e-05 is_feb14
0.01094268439785081 is_march8
0.004835416805536783 is_september1
0.008402272061585902 is_blackFR
0.0 weeklag1
0.15051921760101547 weeklag2
0.12814509336126284 yearlag1


## Prediction

In [168]:
cat.fit(X, y, cat_features) # more iterations?

0:	learn: 195.1393077	total: 68.3ms	remaining: 34.1s
100:	learn: 171.6940928	total: 4.44s	remaining: 17.5s
200:	learn: 166.2999069	total: 8.03s	remaining: 11.9s
300:	learn: 165.5156632	total: 11.3s	remaining: 7.44s
400:	learn: 164.5603469	total: 14.8s	remaining: 3.65s
499:	learn: 164.0361941	total: 18.2s	remaining: 0us


In [169]:
X_res = df[df['is_test']==1].drop(['id', 'demand', 'is_train', 'is_test', 'SKU',
                                          'years'], axis=1)

In [170]:
preds = cat.predict(X_res)
preds = preds.clip(min=0)

In [171]:
smp = pd.read_csv("sample_lt.csv")

In [172]:
ans = []
# check test observation we already had not NaN demand
given_answers = df[(df['is_train']==1)&(df['is_test']==1)] 

for index, idd in enumerate(smp['index'].values):
    if len(given_answers[given_answers['id']==idd].demand.values) != 0: # if for this id demand already exists
        ans.append(given_answers[given_answers['id']==idd].demand.values[0]) # then overwrite the prediction
    else:
        ans.append(preds[index]) # otherwise save prediction

In [173]:
smp['demand'] = ans

In [174]:
smp.to_csv("like+mike+lag+l2.csv", index=False)